In [3]:
import pandas as pd 
import requests 

In [19]:
# read a csv file 
df = pd.read_csv("/Users/kaleemullahqasim/Documents/GitHub/AdIdentifer_Downloader/v0.2/1-4000_only_ad.csv", index_col=0)

In [20]:
df

,url,ads
rank,,
8,youtube.com,True
12,twitter.com,True
20,doubleclick.net,True
34,fastly.net,True
37,bing.com,True
...,...,...
3990,ngs.ru,True
3994,banki.ru,True
3995,dyn.com,True


In [21]:
# add http or https to the urls in df.urls
def add_http(url):
    if url.startswith("https"):
        return url
    else:
        return "https://" + url
    
df["url"] = df["url"].apply(add_http)

In [24]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import requests

def get_status_code(url):
    try:
        response = requests.get(url, allow_redirects=True, timeout=3)
        # Explicitly check for HTTP errors and raise an exception to be caught
        response.raise_for_status()
        return response.status_code
    except requests.exceptions.HTTPError as http_err:
        # Return the HTTP error status code
        return http_err.response.status_code
    except requests.exceptions.RequestException:
        # For non-HTTP errors, return a specific placeholder or code
        return "non_http_error"
    

# Execute the get_status_code function in parallel using ThreadPoolExecutor
# Here, 'max_workers=200' is an arbitrary choice and should be adjusted based on your system and network capacity
with ThreadPoolExecutor(max_workers=100) as executor:
    # Wrap executor.map with tqdm for progress tracking
    # Note: Replace 'df["url"]' with your actual DataFrame column or list of URLs
    status_codes = list(tqdm(executor.map(get_status_code, df["url"]), total=len(df)))
    df["status_code"] = status_codes

# The resulting DataFrame 'df' now includes a 'status_code' column with either HTTP status codes or 'non_http_error' for non-HTTP exceptions.


100%|██████████| 1598/1598 [15:39<00:00,  1.70it/s]


In [25]:
df.status_code.value_counts()

status_code
200               1034
non_http_error     345
403                187
404                  8
429                  6
400                  3
406                  3
401                  3
503                  2
502                  2
501                  1
202                  1
504                  1
423                  1
451                  1
Name: count, dtype: int64

In [27]:
df.status_code.value_counts()

status_code
200               1034
non_http_error     345
403                187
404                  8
429                  6
400                  3
406                  3
401                  3
503                  2
502                  2
501                  1
202                  1
504                  1
423                  1
451                  1
Name: count, dtype: int64

In [26]:
# select the rows with status code 200 and save them to a new csv file
df_200 = df[df["status_code"] == 200]
df_200.to_csv("1-4000-200_only_ad.csv")

In [32]:
# combine two csv files
df1 = pd.read_csv("/Users/kaleemullahqasim/Documents/GitHub/AdIdentifer_Downloader/v0.2/1-4000_only_ad.csv", index_col=0)
df2 = pd.read_csv("/Users/kaleemullahqasim/Documents/GitHub/AdIdentifer_Downloader/v0.2/4000_6800-200websites-ad.csv", index_col=0)

df_combined = pd.concat([df1, df2], ignore_index=True)

In [33]:
df_combined

,url,ads,status_code
0,youtube.com,True,NaN
1,twitter.com,True,NaN
2,doubleclick.net,True,NaN
3,fastly.net,True,NaN
4,bing.com,True,NaN
...,...,...,...
2010,https://unodc.org,True,200.0
2011,https://steelseries.com,True,200.0
2012,https://townnews.com,True,200.0
2013,https://jagran.com,True,200.0


In [34]:
def add_http(url):
    if url.startswith("https"):
        return url
    else:
        return "https://" + url
    
df_combined["url"] = df_combined["url"].apply(add_http)

In [37]:
df_combined

,url,ads,status_code
0,https://youtube.com,True,200
1,https://twitter.com,True,200
2,https://doubleclick.net,True,200
3,https://fastly.net,True,200
4,https://bing.com,True,200
...,...,...,...
2010,https://unodc.org,True,200
2011,https://steelseries.com,True,200
2012,https://townnews.com,True,200
2013,https://jagran.com,True,200


In [36]:
# set status code to 200 for all rows
df_combined["status_code"] = 200

In [38]:
df_combined.to_csv("200_only_ad.csv", index=False)